<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Prepairing" data-toc-modified-id="Prepairing-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Prepairing</a></span><ul class="toc-item"><li><span><a href="#Import-all-necessary" data-toc-modified-id="Import-all-necessary-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Import all necessary</a></span></li><li><span><a href="#Open-and-prepair-the-data" data-toc-modified-id="Open-and-prepair-the-data-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Open and prepair the data</a></span><ul class="toc-item"><li><span><a href="#Clean-up-text-messages-from-unnecessary-garbage" data-toc-modified-id="Clean-up-text-messages-from-unnecessary-garbage-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Clean up text messages from unnecessary garbage</a></span></li><li><span><a href="#Lemmatization" data-toc-modified-id="Lemmatization-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Lemmatization</a></span></li></ul></li><li><span><a href="#Results-of-preparation" data-toc-modified-id="Results-of-preparation-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Results of preparation</a></span></li></ul></li><li><span><a href="#Learning" data-toc-modified-id="Learning-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Learning</a></span><ul class="toc-item"><li><span><a href="#Create-a-list-of-stop-words" data-toc-modified-id="Create-a-list-of-stop-words-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Create a list of stop words</a></span></li><li><span><a href="#Divide-the-data-and-define-features" data-toc-modified-id="Divide-the-data-and-define-features-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Divide the data and define features</a></span><ul class="toc-item"><li><span><a href="#Define-the-features-and-goals-of-learning" data-toc-modified-id="Define-the-features-and-goals-of-learning-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Define the features and goals of learning</a></span></li><li><span><a href="#Divide-the-samples" data-toc-modified-id="Divide-the-samples-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>Divide the samples</a></span></li></ul></li><li><span><a href="#Carry-out-TF-IDF-vectorization" data-toc-modified-id="Carry-out-TF-IDF-vectorization-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Carry out TF-IDF vectorization</a></span></li><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#DesicionTreeClassifier" data-toc-modified-id="DesicionTreeClassifier-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>DesicionTreeClassifier</a></span></li><li><span><a href="#CatBoostClassifier" data-toc-modified-id="CatBoostClassifier-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>CatBoostClassifier</a></span></li><li><span><a href="#Intermediate-conclusion" data-toc-modified-id="Intermediate-conclusion-2.7"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>Intermediate conclusion</a></span></li></ul></li><li><span><a href="#Model-testing" data-toc-modified-id="Model-testing-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Model testing</a></span></li><li><span><a href="#Conclusions" data-toc-modified-id="Conclusions-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Conclusions</a></span></li></ul></div>

# Toxic tweets

Online store "Wikishop" launches a new service. Now users can edit and supplement product descriptions, just like in wiki communities. That is, clients propose their edits and comment on the changes of others. The store needs a tool that will look for toxic comments and submit them for moderation.

Train the model to classify comments into positive and negative. At your disposal is a dataset with markup on the toxicity of edits.

Build a model with a quality metric *F1* of at least 0.75.

## Prepairing


### Import all necessary

In [1]:
import pandas as pd
import re
import spacy
nlp = spacy.load('en_core_web_sm')
from dframcy import DframCy
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings('ignore')

state = 12345

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pgore\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Open and prepair the data

In [2]:
data = pd.read_csv ('toxic_comments.csv')
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


Data Description:
1. Unnamed: 0 - column with old indexes from csv or excell file, can be deleted
2. text - a column with text messages, there are capital letters and separators, this should also be removed.
3. toxic - a column indicating whether the comment is toxic or not, is the target feature.

Delete column Unnamed:0

In [3]:
data = data.drop(columns = ['Unnamed: 0'],axis = 1)
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


#### Clean up text messages from unnecessary garbage

In [4]:
def clear_text(text):
    clear_text = re.sub(r'[^a-zA-Z]', ' ', text) 
    clear_text = clear_text.split()
    clear_text = " ".join(clear_text)
    clear_text = clear_text.lower()
    return clear_text


data['clear_text'] = data['text'].apply(clear_text)

In [5]:
data.head()

,text,toxic,clear_text
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,d aww he matches this background colour i m se...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not trying to edit war it s...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i can t make any real suggestions on impr...
4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember wh...


#### Lemmatization

In [6]:
dframcy = DframCy(nlp)

def lemmatize (text):
    doc = dframcy.nlp (text)
    #doc_df = dframcy.to_dataframe(doc)
    #doc_df['index'] = text[0]
    lemm_text = " ".join([token.lemma_ for token in doc])
    return lemm_text
    


In [7]:
corpus = data['clear_text'].copy()
corpus[0] = lemmatize(corpus[0])
corpus

0         explanation why the edit make under my usernam...
1         d aww he matches this background colour i m se...
2         hey man i m really not trying to edit war it s...
3         more i can t make any real suggestions on impr...
4         you sir are my hero any chance you remember wh...
                                ...                        
159287    and for the second time of asking when your vi...
159288    you should be ashamed of yourself that is a ho...
159289    spitzer umm theres no actual article for prost...
159290    and it looks like it was actually you who put ...
159291    and i really don t think you understand i came...
Name: clear_text, Length: 159292, dtype: object

In [8]:
data['lemm_text'] = pd.Series(corpus)
data

,text,toxic,clear_text,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...,explanation why the edit make under my usernam...
1,D'aww! He matches this background colour I'm s...,0,d aww he matches this background colour i m se...,d aww he matches this background colour i m se...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not trying to edit war it s...,hey man i m really not trying to edit war it s...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i can t make any real suggestions on impr...,more i can t make any real suggestions on impr...
4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember wh...,you sir are my hero any chance you remember wh...
...,...,...,...,...
159287,""":::::And for the second time of asking, when ...",0,and for the second time of asking when your vi...,and for the second time of asking when your vi...
159288,You should be ashamed of yourself \n\nThat is ...,0,you should be ashamed of yourself that is a ho...,you should be ashamed of yourself that is a ho...
159289,"Spitzer \n\nUmm, theres no actual article for ...",0,spitzer umm theres no actual article for prost...,spitzer umm theres no actual article for prost...
159290,And it looks like it was actually you who put ...,0,and it looks like it was actually you who put ...,and it looks like it was actually you who put ...


### Results of preparation

1. Removed extra column.
2. Texts are cleared of unnecessary characters and capital letters.
3. Texts are lemmatized and prepared for training the model.

## Learning

### Create a list of stop words

In [9]:
stopwords = set(nltk_stopwords.words('english'))

### Divide the data and define features

#### Define the features and goals of learning

In [10]:
target = data['toxic']
features = data['lemm_text']

#### Divide the samples

In [11]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size = 0.2, shuffle = True, random_state = state)

features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid, target_valid, shuffle = True, test_size = 0.5, random_state = state)

features_train.shape[0], features_valid.shape[0], features_test.shape[0]

(127433, 15929, 15930)

### Carry out TF-IDF vectorization

In [12]:
tf_idf = TfidfVectorizer(stop_words = stopwords)

In [13]:
train_tf_idf = tf_idf.fit_transform(features_train)
valid_tf_idf = tf_idf.transform(features_valid)
test_tf_idf = tf_idf.transform(features_test)

### LogisticRegression

In [14]:
LR = LogisticRegression(max_iter = 1000, class_weight = 'balanced', C = 10)
LR.fit(train_tf_idf, target_train)
LR_pred_valid = LR.predict(valid_tf_idf)
LR_F1 = f1_score(target_valid, LR_pred_valid)
print ('f1_score on LR =', round (LR_F1,3))

f1_score on LR = 0.778


### DesicionTreeClassifier

In [15]:
DTC = DecisionTreeClassifier(random_state = state, class_weight = 'balanced')
DTC.fit(train_tf_idf, target_train)
DTC_pred_valid = DTC.predict(valid_tf_idf)
DTC_F1 = f1_score(target_valid, DTC_pred_valid)
print ('f1_score on DTC =', round (DTC_F1,3))

f1_score on DTC = 0.626


### CatBoostClassifier

In [16]:
CBC = CatBoostClassifier(iterations = 50, learning_rate = 0.5, 
                           verbose = 5, random_state = state)
CBC.fit(train_tf_idf, target_train)


0:	learn: 0.3450303	total: 1.08s	remaining: 52.8s
5:	learn: 0.2066477	total: 5.48s	remaining: 40.2s
10:	learn: 0.1845820	total: 9.93s	remaining: 35.2s
15:	learn: 0.1727016	total: 14.3s	remaining: 30.3s
20:	learn: 0.1651376	total: 18.7s	remaining: 25.8s
25:	learn: 0.1583897	total: 23.3s	remaining: 21.5s
30:	learn: 0.1525723	total: 27.7s	remaining: 17s
35:	learn: 0.1480318	total: 32.1s	remaining: 12.5s
40:	learn: 0.1439869	total: 36.5s	remaining: 8.01s
45:	learn: 0.1407948	total: 40.8s	remaining: 3.55s
49:	learn: 0.1377563	total: 44.2s	remaining: 0us


In [17]:
CBC_pred_valid = CBC.predict(valid_tf_idf)
CBC_F1 = f1_score(target_valid, CBC_pred_valid)
print ('f1_score on CatBoost =', round (CBC_F1,3))

f1_score on CatBoost = 0.74


### Intermediate conclusion

Logistic regression was chosen as the best model, since its F1 metric is greater than the lower threshold of 0.76 and the training time is the shortest.

## Model testing

Check the chosen model on the test sample

In [18]:
LR_pred_test = LR.predict(test_tf_idf)
LR_F1_test = f1_score(target_test, LR_pred_test)
print ('f1_score =', round (LR_F1_test,3))


f1_score = 0.771


## Conclusions

1. The data was prepared, it was cleaned from unnecessary characters and lemmatization was carried out.
2. A list of stop words is created, the data is divided into learning features and a target feature.
3. TF-IDF vectorization of features for training was carried out.
4. Trained the Logistic Regression model. F1 on the validation set - 0.778.
5. The classification decision tree model was trained. F1 on the validation selection - 0.626
6. Trained the CatBoostClassifier model. F1 on the validation set is 0.74, but the training time (compared to Regression) is very long.
7. The Logistic Regression model was recognized as the best model.
8. F1 of the best model on the test set is 0.771.